<a href="https://colab.research.google.com/github/ghlai9665/transformer-implementation/blob/main/Transformer_Implementation_PyTorch_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies


In [1]:
!pip install torch numpy matplotlib spacy torchtext==0.4.0 seaborn

     |████████████████████████████████| 61kB 3.1MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [2]:
# import dependencies
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn
import time
seaborn.set_context(context="talk")
%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Load Data

In [4]:
??torch.add

In [5]:
!python -m spacy download en
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 709kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=b5aeed4aad98f0784de5545f40dca0e1aa6074442ec4379b328fb756847e1c64
  Stored in directory: /tmp/pip-ephem-wheel-cache-huucwbi_/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [6]:
from torchtext.datasets import IWSLT
from torchtext.data import Field, BucketIterator

# define the tokenizers and params for the data (i don't think Spacy does subword tokenization)
SRC = Field(tokenize = "spacy",
            tokenizer_language = "de",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)
TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

# split data into train, validate, and test group, all are torch.data.Dataset objects
train_data, valid_data, test_data = IWSLT.splits(exts = ('.de', '.en'),
                                                 fields = (SRC, TRG))

downloading de-en.tgz


de-en.tgz: 100%|██████████| 24.2M/24.2M [00:11<00:00, 2.04MB/s]


.data/iwslt/de-en/IWSLT16.TED.tst2011.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2014.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2013.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2014.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2013.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2012.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2014.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2010.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.dev2012.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2014.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2013.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2013.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2011.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2010.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2012.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TEDX.dev2012.de-en.de.xml
.data/iwslt/de-en/train.tags.de-en.en
.data/iwslt/de-en/train.tags.de-en.de


In [7]:
import torchtext
import math

In [8]:
print(type(train_data))
print(isinstance(train_data, torchtext.data.Example)) 
print(isinstance(train_data, torchtext.data.Dataset)) 
print("IWSLT type: ", type(IWSLT))
print(isinstance(IWSLT, torchtext.data.Dataset)) 
print(train_data[0])
print(isinstance(train_data[0], torchtext.data.Example)) 
print(train_data[0].__dict__.keys())
print(train_data[0].src)
print(train_data[0].trg)
print(train_data[1].trg)

'''
From the print statements, you can see that train_data is a `torchtext.data.Dataset` object, which is a "list" containing many
`torchtext.data.example.Example` objects!
'''

torchtext.datasets.translation.IWSLT
False
True
IWSLT type:  <class 'typing.GenericMeta'>
False
True
dict_keys(['src', 'trg'])
['david', 'gallo', ':', 'das', 'ist', 'bill', 'lange', '.', 'ich', 'bin', 'dave', 'gallo', '.']
['david', 'gallo', ':', 'this', 'is', 'bill', 'lange', '.', 'i', "'m", 'dave', 'gallo', '.']
['and', 'we', "'re", 'going', 'to', 'tell', 'you', 'some', 'stories', 'from', 'the', 'sea', 'here', 'in', 'video', '.']


'\nFrom the print statements, you can see that train_data is a `torchtext.data.Dataset` object, which is a "list" containing many\n`torchtext.data.example.Example` objects!\n'

In [9]:
print("len(train_data): ", len(train_data))
print("get item: ", train_data[1])
print("get item: ", train_data.__getitem__(1))

len(train_data):  196884
get item:  <torchtext.data.example.Example object at 0x7f110171feb8>
get item:  <torchtext.data.example.Example object at 0x7f110171feb8>


In [10]:
# build a vocabulary to index words with numbers; you can go from word to number via SRC.vocab.stoi
# Batch objects contains references to Dataset objects which itself contains references to Field objects. 
# When Batch is initialized, it calls Field.process which calls Field.numericalize which calls self.vocab -- if you don't build the vocab here, you will get an error when Batch is initialized later
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [11]:
BATCH_SIZE = 4
# BucketIterator batches sentences of similar lengths together to minimize padding
split_result = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE)
print(split_result)
print(type(split_result))
# One BucketIterator for each dataset
train_iterator, valid_iterator, test_iterator = split_result

(<torchtext.data.iterator.BucketIterator object at 0x7f10dc9221d0>, <torchtext.data.iterator.BucketIterator object at 0x7f10dc9228d0>, <torchtext.data.iterator.BucketIterator object at 0x7f10dc922978>)
<class 'tuple'>


In [12]:
print(train_iterator.dataset[0].src)
print(train_iterator)

['david', 'gallo', ':', 'das', 'ist', 'bill', 'lange', '.', 'ich', 'bin', 'dave', 'gallo', '.']


In [13]:
# size of the source vocab
src_vocab_size = len(SRC.vocab.itos)
tgt_vocab_size = len(TRG.vocab.itos)
# print(src_vocab_size)
# print(tgt_vocab_size)

In [14]:
# translate batch.srcs or batch.nums to sentence form
def batch_nums_to_sentences(nums, src):
  nums_transposed = nums.transpose(0,1)
  m, n = nums_transposed.shape
  # print("The transposed shape is: ", m, " * ", n)
  batch_tokens = [[None for _ in range(n)] for _ in range(m)]
  for i in range(m):
    for j in range(n):
      batch_tokens[i][j] = SRC.vocab.itos[nums_transposed[i][j]] if src else TRG.vocab.itos[nums_transposed[i][j]]

  batch_sentences = [" ".join(row) for row in batch_tokens]
  # print("there are ", len(batch_sentences), " sentences in the batch.")

  return batch_sentences


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
counter = 0
for i, batch in enumerate(train_iterator):

  if counter > 2:
    break
  src, trg = batch.src.to(device=device), batch.trg.to(device=device)
  print("batch.src.device: ", batch.src.device)
  print("src.device: ", src.device)
  print("batch.trg.device: ", batch.trg.device)
  print("trg.device: ", trg.device)
  
  # # ***** the size of torch.data.batch.src/trg is (sentence length, batch_size) ****
  # print(batch.trg.shape)
  # print(batch_nums_to_sentences(batch.trg, False))
  # print(batch_nums_to_sentences(batch.src, True))

  counter += 1

batch.src.device:  cpu
src.device:  cuda:0
batch.trg.device:  cpu
trg.device:  cuda:0
batch.src.device:  cpu
src.device:  cuda:0
batch.trg.device:  cpu
trg.device:  cuda:0
batch.src.device:  cpu
src.device:  cuda:0
batch.trg.device:  cpu
trg.device:  cuda:0


# Define Functions for Training

In [19]:
def embed(x, vocab_size, d_model=512):
  # two embedding (1) each token's numeral value is mapped to a embedding vector (2) positional embedding is applied
  number_to_embedding = nn.Embedding(vocab_size, d_model)
  pos_embedding = PositionalEncoding(d_model)

  x = number_to_embedding(x) * math.sqrt(d_model)
  x = pos_embedding(x)
  return x

In [17]:
class Batch:
  def __init__(self, src, trg=None, pad_value=0):
    # input src & trg are shape (sentence_len, batch_size) and embedded into (sentence_len, batch_size, d_model)
    self.src = embed(src, src_vocab_size)
    if trg != None:
      # given src & trg_x, we try to predict trg_y, which has ntokens words (i.e. we make ntokens predictions)
      trg_embedding = embed(trg, tgt_vocab_size)
      self.trg_x = trg_embedding[:-1, :, :]
      self._trg_y = trg[1:, :]
      self.trg_y = _trg_y.reshape(-1)
      self.ntokens = (_trg_y != pad_value).sum()

In [18]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [21]:
class Generator(nn.Module):
  def __init__(self, d_model, vocab):
    super().__init__()
    self.proj = nn.Linear(d_model, vocab) # define the linear projection, which takes d_model sized activation and output linear mapping to assign probability to each of the vocab
  
  def forward(self, x):
    return F.log_softmax(self.proj(x), dim=-1)

In [ ]:
class Log:
  def __init__(self):
    self.start = time.time()
    self.total_loss = 0
    self.total_tokens = 0
    # loss & tokens accumulated over the last < 50 steps
    self.cur_loss = 0
    self.cur_tokens = 0 

  def batch_info(batch, loss):
    self.total_loss += loss
    self.total_tokens += int(batch.ntokens)
    self.cur_loss += loss
    self.cur_tokens += int(batch.ntokens)
  
  def reset_every_50batches():
    self.cur_tokens = 0
    self.cur_loss = 0
    self.start = time.time()

  def show_every_50batches(i):
    elapsed = time.time() - self.start
    print("-----")
    print("Epoch step: %d  Average Loss Per Token Over the Last 50 Batch: %f  Processed Tokens per Sec %f" % (i, self.cur_loss / self.cur_tokens, self.cur_tokens / elapsed))
    self.reset_every_50batches()

  def show_epoch():
    print("Total loss for the epoch is: ", total_loss / total_tokens)

In [22]:
# Every epoch is an iteration over the entire training set (how many steps are in one epoch depends on the batch_size)
def train_epoch(data, model, hyper_params):
  model.train() 
  # varialbes for logging
  log = Log()
  # train the model batch-by-batch 
  for i, batch in enumerate(data):
    batch_loss = train_batch(batch, model, hyper_params)
    log.batch_info(batch, batch_loss)
    if i % 50 == 1:
      log.show_every_50batches(i)
  log.show_epoch()

In [ ]:
def train_batch(batch, model, hyper_params):
    def forward_pass():
      src = batch.src.to(device=device)
      trg_x = batch.trg_x.to(device=device)
      pred = model(src, trg_x)
      return pred
    
    def calculate_loss(pred):
      trg_y = batch.trg_y.to(device=device)
      loss = hyper_params.criterion(pred.view(-1,tgt_vocab_size), trg_y)  
      return loss

    pred = foward_pass()
    loss = calculate_loss(pred)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    return float(loss)

# Define Model


In [ ]:
class Model:
  def __init__(self, encoder_decoder, generator):
    self.encoder_decoer = encoder_decoder
    self.generator = generator

  def forward(src, trg_x):
    out = self.encoder_decoder.foward(src, trg_x)
    out = self.generator.forward(out)
    return out

In [ ]:
class HyperParams:
  def __init__(self, criterion = None, optimizer = None, scheduler = None):
    self.criterion, self.optimizer, self.scheduler = criterion, optimizer, scheduler

In [24]:
# variables
pad_value = TRG.vocab.stoi['<pad>']
d_model = 512
lr = 5.0 

# initialize the training params
data = (Batch(batch.src, batch.trg, pad_value) for batch in train_iterator)

# initialize model
encoder_decoder = nn.Transformer().to(device)
generator = Generator(d_model, tgt_vocab_size).to(device)
model = Model(encoder_decoder, generator)

hyper_params = HyperParams(criterion = nn.CrossEntropyLoss(), optimizer = torch.optim.SGD(model.parameters(), lr=lr), scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95))

# Start Training

In [ ]:
# train for 10 epochs
for epoch in range(1):
  print("------------Training epoch ", epoch, "--------------")
  train_epoch(data, model, hyper_params)
  scheduler.step()
  # set to eval model to check how good the model is after each loop
  # print(train_epoch((batchify(b, pad_idx, device) for b in valid_iterator), model, criterion, epoch))